# Data Cleaning!


In [282]:
import janitor
import pandas as pd
import ydata_profiling as yp
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [283]:
def one_hot_encode(df):
    """
    One-hot encodes the categorical columns of a DataFrame.

    Args:
        df (pd.DataFrame): The DataFrame to be one-hot encoded.

    Returns:
        pd.DataFrame: The one-hot encoded DataFrame.
    """
    # Identify categorical columns
    categorical_cols = df.select_dtypes(include=["object", "category"]).columns
    non_categorical_cols = df.select_dtypes(exclude=["object", "category"]).columns

    # One-hot encode categorical columns
    encoder = OneHotEncoder()

    encoded_array = encoder.fit_transform(df[categorical_cols]).toarray()

    encoded_df = pd.DataFrame(
        encoded_array, columns=encoder.get_feature_names_out(categorical_cols)
    )

    result_df = pd.concat(
        [df[non_categorical_cols].reset_index(drop=True), encoded_df], axis=1
    )

    return result_df

In [284]:
def standardise(df, non_categorical_cols):
    """
    One-hot encodes the categorical columns of a DataFrame.

    Args:
        df (pd.DataFrame): The DataFrame to be one-hot encoded.

    Returns:
        pd.DataFrame: The one-hot encoded DataFrame.
    """

    # Standardise categorical columns
    encoder = StandardScaler()

    encoded_array = encoder.fit_transform(df[non_categorical_cols])

    encoded_df = pd.DataFrame(
        encoded_array, columns=encoder.get_feature_names_out(non_categorical_cols)
    )

    all_cols_set = set(df.columns)
    non_categorical_cols_set = set(non_categorical_cols)
    categorical_cols = all_cols_set - non_categorical_cols_set

    result_df = pd.concat(
        [
            df[list(categorical_cols)].reset_index(drop=True),
            encoded_df,
        ],
        axis=1,
    )

    # reorder columns to have unique_id at the front
    non_pid_cols = [col for col in result_df.columns if col != "pid"]
    result_df = result_df[["pid"] + non_pid_cols]
    return result_df

In [285]:
ames_df = pd.read_csv("data/AmesHousing.csv")
no_csv = pd.read_csv("data/na_list.csv")

### 1.) Replace meaningful Nans with pair in na_list.csv

In [286]:
replace_dict = dict(zip(no_csv["feature"], no_csv["NA meaning"]))

ames_df = ames_df.fillna(value=replace_dict, axis=0)

### 2.) Correct column types

In [287]:
ames_df = pd.concat(
    [
        ames_df.select_dtypes(None, ["object"]),
        ames_df.select_dtypes(["object"]).apply(pd.Series.astype, dtype="category"),
    ],
    axis=1,
).reindex(ames_df.columns, axis=1)

### 3.) Remove scewed, uninformative columns

In [288]:
ames_df = ames_df.clean_names(remove_special=True)

column_to_remove_list = [
    "alley",
    "land_slope",
    "condition_2",
    "roof_matl",
    "mas_vnr_type",
    "mas_vnr_type",
    "mas_vnr_area",
    "bsmtfin_type_2",
    "heating",
    "low_qual_fin_sf",
    "enclosed_porch",
    "3ssn_porch",
    "screen_porch",
    "pool_area",
    "misc_feature",
    "misc_val",
    "mo_sold",
    "yr_sold",
    "sale_type",
    "sale_condition",
    "order",
]

ames_df = ames_df.drop(columns=column_to_remove_list).reset_index(drop=True)

### 4.) bad categories get cleaned

In [289]:
ames_df["ms_zoning"] = ames_df["ms_zoning"].apply(lambda x: x.split(" ")[0])

### 5.) one hot encoding

In [290]:
ames_df = one_hot_encode(ames_df)

### 6.) standardising


In [291]:
non_categorical_cols = ames_df.select_dtypes(exclude=["object", "category"]).columns
non_categorical_cols = non_categorical_cols.drop(["pid"])

ames_df = standardise(ames_df, non_categorical_cols)

#### BEFORE INTERPOLATION Test train split (including validation sets)


In [292]:
# type of test train split
no_test_train = False
normal_test_train = False
test_train_with_val = True

In [293]:
if no_test_train:
    # drop lot_frontage due to data leakage
    ames_df = ames_df.drop("lot_frontage", axis=1)

In [294]:
# split into X and y dataframes
X = ames_df.drop("saleprice", axis=1)
y = ames_df[["pid", "saleprice"]]

In [295]:
if test_train_with_val:
    X_train, X_temp, y_train, y_temp = train_test_split(
        X, y, test_size=0.8, random_state=42
    )

    # split temp into 75/25 split
    X_val1, X_temp, y_val1, y_temp = train_test_split(
        X_temp, y_temp, test_size=0.75, random_state=42
    )

    # next split is 2/3 (60 percent of dataset left)
    X_val2, X_temp, y_val2, y_temp = train_test_split(
        X_temp, y_temp, test_size=(2 / 3), random_state=42
    )

    # last split is 50/50 (40 percent of dataset left)
    X_val3, X_val4, y_val3, y_val4 = train_test_split(
        X_temp, y_temp, test_size=0.5, random_state=42
    )

    # create dictionaries to loop through processes

    X_data_dict = {
        "train": X_train,
        "val1": X_val1,
        "val2": X_val2,
        "val3": X_val3,
        "val4": X_val4,
    }

    y_data_dict = {
        "train": y_train,
        "val1": y_val1,
        "val2": y_val2,
        "val3": y_val3,
        "val4": y_val4,
    }

In [296]:
if normal_test_train:
    # 80/20 test/train split
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.8, random_state=42
    )
    X_data_dict = {"train": X_train, "test": X_test}
    y_data_dict = {"train": y_train, "test": y_test}

In [297]:
if no_test_train:
    X_data_dict = {"train": X}
    y_data_dict = {"train": y}

### 5.) Interpolate columns

In [298]:
def interpolation_steps(df):
    try:
        df["lot_frontage"] = df["lot_frontage"].fillna(df["lot_frontage"].mean())
    except KeyError as e:
        return df
    return df

### 6.) One hot encoding categorical data

In [299]:
# find a list of non categorical columns
# non_categorical_cols = ames_df.select_dtypes(exclude=["object", "category"]).columns
# non_categorical_cols = non_categorical_cols.drop(["pid", "saleprice"])

# apply one hot encoding to all datasets
for key, value in X_data_dict.items():
    print(
        f"###################{key}###################\nBefore\n{key}:{X_data_dict[key].shape}"
    )
    X_data_dict[key] = interpolation_steps(X_data_dict[key])
    print(f"\nAfter interpolation\n{key}:{X_data_dict[key].shape}")
    # X_data_dict[key] = one_hot_encode(X_data_dict[key])
    # print(f"\nAfter 1h\n{key}:{X_data_dict[key].shape}")
    # X_data_dict[key] = standardise(X_data_dict[key], non_categorical_cols)
    # y_data_dict[key] = standardise(y_data_dict[key], ["saleprice"])
    # print(f"\nAfter standardisation\n{key}:{X_data_dict[key].shape}")

for key, value in X_data_dict.items():

    X_data_dict[key].to_csv(f"data/X_{key}.csv", index=False)

    y_data_dict[key].to_csv(f"data/y_{key}.csv", index=False)

###################train###################
Before
train:(586, 249)

After interpolation
train:(586, 249)
###################val1###################
Before
val1:(586, 249)

After interpolation
val1:(586, 249)
###################val2###################
Before
val2:(586, 249)

After interpolation
val2:(586, 249)
###################val3###################
Before
val3:(586, 249)

After interpolation
val3:(586, 249)
###################val4###################
Before
val4:(586, 249)

After interpolation
val4:(586, 249)


# Create a massive report

In [300]:
"""profile = yp.ProfileReport(ames_df)
profile.to_notebook_iframe()
profile.to_file("eda_report.html")"""

'profile = yp.ProfileReport(ames_df)\nprofile.to_notebook_iframe()\nprofile.to_file("eda_report.html")'